# 3.3 线性回归的简洁实现

In [1]:
import torch
from torch import nn
import numpy as np
torch.manual_seed(1)

print(torch.__version__)
torch.set_default_tensor_type('torch.FloatTensor')

1.7.1


## 3.3.1 生成数据集

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)


## 3.3.2 读取数据

In [20]:
import torch.utils.data as Data

batch_size = 10

# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
print(dataset[0:5])  # 返回一个tuple。第一项是features，第二项是labels

# 把 dataset 放入 DataLoader
data_iter = Data.DataLoader(
    dataset=dataset,      # torch TensorDataset format
    batch_size=batch_size,      # mini batch size
    shuffle=True,               # 要不要打乱数据 (打乱比较好)
    num_workers=2,              # 多线程来读数据
)

(tensor([[-1.3129, -1.2917],
        [ 0.1607, -0.0704],
        [-0.5030, -0.4178],
        [-0.7095, -0.7497],
        [-0.0150,  0.9898]]), tensor([5.9482, 4.7629, 4.6191, 5.3160, 0.7978]))


In [22]:
for X, y in data_iter:
    print(X, '\n', y)
    break

tensor([[-1.6704,  0.8601],
        [ 0.8138, -0.2915],
        [ 2.2697, -1.1882],
        [ 0.6173, -0.4717],
        [ 0.6607,  1.6692],
        [-0.0493, -1.5546],
        [ 0.1954, -1.0250],
        [-1.4442,  1.5956],
        [ 0.9169,  0.3599],
        [ 0.3161, -1.4369]]) 
 tensor([-2.0606,  6.8242, 12.7736,  7.0454, -0.1583,  9.3865,  8.0737, -4.1144,
         4.8011,  9.6895])


## 3.3.3 定义模型

In [5]:
class LinearNet(nn.Module):
    # 继承nn.Module
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)

    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net) # 使用print可以打印出网络的结构

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [6]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
    )

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict
net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
        ]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [7]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[0.6873, 0.2890]], requires_grad=True)
Parameter containing:
tensor([-0.1508], requires_grad=True)


## 3.3.4 初始化模型参数

In [8]:
from torch.nn import init

init.normal_(net[0].weight, mean=0.0, std=0.01)
init.constant_(net[0].bias, val=0.0)  # 也可以直接修改bias的data: net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

In [9]:
for param in net.parameters():
    print(param)

Parameter containing:
tensor([[ 0.0100, -0.0072]], requires_grad=True)
Parameter containing:
tensor([0.], requires_grad=True)


## 3.3.5 定义损失函数

In [10]:
loss = nn.MSELoss()

## 3.3.6 定义优化算法

In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [12]:
# 为不同子网络设置不同的学习率
# optimizer =optim.SGD([
#                 # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#                 {'params': net.subnet1.parameters()}, # lr=0.03
#                 {'params': net.subnet2.parameters(), 'lr': 0.01}
#             ], lr=0.03)

In [13]:
# # 调整学习率
# for param_group in optimizer.param_groups:
#     param_group['lr'] *= 0.1 # 学习率为之前的0.1倍

## 3.3.7 训练模型

In [14]:
num_epochs = 3
for epoch in range(1, num_epochs + 1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' % (epoch, l.item()))

epoch 1, loss: 0.000523
epoch 2, loss: 0.000060
epoch 3, loss: 0.000098


In [15]:
dense = net[0]
print(true_w, dense.weight.data)
print(true_b, dense.bias.data)

[2, -3.4] tensor([[ 1.9997, -3.4008]])
4.2 tensor([4.2002])
